In [16]:
import math
from scripts_stock.cfg.out_file_name import OutFileName
from scripts_stock.cfg.set_dir import ProjectDir
from scripts_stock.cfg.stock_list import *
from scripts_stock.utils.analysis.stock_stat_index import df_to_stock_df, stock_kdj
import os
from scripts_stock.utils.common import CommonScript
from scripts_stock.utils.string_process import StringProcess
from scripts_stock.data_base.get_table_info import GetDataFromDB

from scripts_stock.analysis.fuquan_data.get_fuquan_index_v2 import *
from datetime import datetime

In [2]:

def get_weekly_price(input_df: pd.DataFrame):
    df1 = input_df
    df2 = df1.groupby(lambda x:math.floor(x/5)).min()
    df3 = df1.groupby(lambda x:math.floor(x/5)).max()
    df4 = df1.groupby(lambda x:math.floor(x/5)).last()
    df5 = df1.groupby(lambda x:math.floor(x/5)).first()
    df2["high"]=df3["high"]
    df2["close"] = df4["close"]
    df2["open"] = df5["open"]
    df2['stock_date'] = df3["date"]
    return df2

In [3]:
def get_stock_indicator(stock_df_in, if_print=False):
    stock_df = df_to_stock_df(
        stock_df_in[["open", "close", "high", "low", "stock_index", "stock_date"]])
    stock_kdj_ind, _ = stock_kdj(stock_df)
    if if_print:
        print(stock_kdj_ind)
    # stock_kdj_ind["stock_index"] = stock_df_in["stock_index"].values[0]
    return stock_kdj_ind

In [6]:
stock_index_v1 = StockIndex(ProjectDir)

In [14]:
df_raw = GetDataFromDB.get_fuquan_all_one_stock_df(600025)
start_from = df_raw.shape[0]%5
df1 = df_raw.iloc[start_from:].copy(
    deep=False).reset_index().sort_values("date")
df_weekly = stock_index_v1.get_weekly_price(df1)
# stock_kdj_ind = get_stock_indicator(df_weekly)
# stock_kdj_ind

In [32]:

def get_weekly_price_v2(input_df: pd.DataFrame):
    df1 = input_df
    df2 = df1.groupby("week_group").min()
    df3 = df1.groupby("week_group").max()
    df4 = df1.groupby("week_group").last()
    df5 = df1.groupby("week_group").first()
    df2["high"] = df3["high"]
    df2["close"] = df4["close"]
    df2["open"] = df5["open"]
    df2['stock_date'] = df3["date"]
    return df2

In [39]:
df3 = df_raw.groupby("week_group").max()
df3

,date,open,close,high,low,transaction_volume,stock_index,update_time,datetime
week_group,,,,,,,,,
2017-12-15,2017-12-15,1.38,1.90,1.90,1.38,1.006897e+07,600025,2025-01-03 06:09:31.993927,2017-12-15
2017-12-22,2017-12-22,3.38,3.81,3.81,3.35,3.381055e+09,600025,2025-01-03 06:09:31.993927,2017-12-22
2017-12-29,2017-12-29,4.89,4.90,4.90,4.53,4.833708e+09,600025,2025-01-03 06:09:31.993927,2017-12-29
2018-01-05,2018-01-05,4.05,4.13,4.27,3.98,2.744447e+09,600025,2025-01-03 06:09:31.993927,2018-01-05
2018-01-12,2018-01-12,3.96,3.96,4.02,3.88,1.341450e+09,600025,2025-01-03 06:09:31.993927,2018-01-12
...,...,...,...,...,...,...,...,...,...
2024-12-06,2024-12-06,9.64,9.64,9.65,9.50,3.889980e+08,600025,2025-01-03 06:09:31.993927,2024-12-06
2024-12-13,2024-12-13,9.66,9.67,9.76,9.62,3.966351e+08,600025,2025-01-03 06:09:31.993927,2024-12-13
2024-12-20,2024-12-20,9.81,9.81,9.95,9.70,5.647233e+08,600025,2025-01-03 06:09:31.993927,2024-12-20


In [35]:
df_raw = GetDataFromDB.get_fuquan_all_one_stock_df(600025)
df_raw['datetime'] = pd.to_datetime(df_raw['date'])
df_raw['week_group'] = df_raw['datetime'].dt.to_period(
    'W-FRI').apply(lambda r: r.end_time.strftime("%Y-%m-%d"))
# 添加一列表示星期几 (0=Monday, 6=Sunday)
df3 = get_weekly_price_v2(df_raw)

get_stock_indicator(df3)

NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values


,date,open,close,high,low,stock_index,date_time,stock_date,macdh,cci,...,rsi_24,kdjk,kdjd,kdjj,boll_ub,boll_lb,macd,macds,wr_6,wr_10
0,2017-12-15,1.38,1.90,1.90,1.38,600025,2017-12-15,2017-12-15,0.000000,NaN,...,NaN,66.666667,55.555556,88.888889,NaN,NaN,0.000000,0.000000,0.000000,0.000000
1,2017-12-22,2.21,3.81,3.81,2.21,600025,2017-12-22,2017-12-22,0.019046,66.666667,...,100.000000,77.777778,62.962963,107.407407,5.556148,0.153852,0.042853,0.023807,0.000000,0.000000
2,2017-12-29,4.25,4.08,4.90,3.79,600025,2017-12-29,2017-12-29,0.023116,86.029412,...,100.000000,77.420034,67.781987,96.696128,5.640082,0.886585,0.062975,0.039860,23.295455,23.295455
3,2018-01-05,3.99,4.00,4.27,3.80,600025,2018-01-05,2018-01-05,0.018407,57.171923,...,96.177551,76.423962,70.662645,87.946596,5.523225,1.371775,0.067696,0.049289,25.568182,25.568182
4,2018-01-12,3.91,3.60,4.02,3.59,600025,2018-01-12,2018-01-12,0.002550,30.722817,...,80.185907,71.972035,71.099108,73.717889,5.280798,1.675202,0.052919,0.050369,36.931818,36.931818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2024-12-06,9.32,9.43,9.65,9.27,600025,2024-12-06,2024-12-06,-0.238029,-99.419384,...,46.713470,5.218216,7.876770,-0.098892,12.308903,8.857097,-0.201935,0.036094,77.551020,93.313070
357,2024-12-13,9.44,9.62,9.76,9.35,600025,2024-12-13,2024-12-13,-0.200130,-71.801761,...,48.278188,12.470039,9.407860,18.594397,12.179816,8.782184,-0.214067,-0.013938,48.101266,87.537994
358,2024-12-20,9.62,9.52,9.95,9.51,600025,2024-12-20,2024-12-20,-0.172138,-53.639847,...,47.511972,16.201400,11.672373,25.259454,12.041706,8.712294,-0.229111,-0.056972,60.759494,79.605263
359,2024-12-27,9.51,9.72,9.92,9.33,600025,2024-12-27,2024-12-27,-0.132287,-48.055609,...,49.194733,28.147872,17.164206,50.115204,11.913050,8.674950,-0.222332,-0.090044,31.081081,61.068702


In [23]:
date_object = datetime.strptime("2025-01-05", "%Y-%m-%d")

# 获取星期几，使用 strftime 方法返回中文的星期几
day_of_week = date_object.weekday()+1
day_of_week

7

In [15]:
df_weekly

,index,date,open,close,high,low,transaction_volume,stock_index,update_time,stock_date
0,2,2017-12-19,2.55,3.83,4.31,2.55,8.929354e+06,600025,2025-01-03 06:09:31.993927,2017-12-25
1,7,2017-12-26,3.92,4.12,4.90,3.79,1.847257e+09,600025,2025-01-03 06:09:31.993927,2018-01-02
2,12,2018-01-03,4.03,3.90,4.20,3.80,9.743760e+08,600025,2025-01-03 06:09:31.993927,2018-01-09
3,17,2018-01-10,3.85,3.81,3.88,3.28,9.417274e+08,600025,2025-01-03 06:09:31.993927,2018-01-16
4,22,2018-01-17,4.06,3.70,4.23,3.59,5.837654e+08,600025,2025-01-03 06:09:31.993927,2018-01-23
...,...,...,...,...,...,...,...,...,...,...
337,1687,2024-11-29,9.35,9.41,9.65,9.27,1.943787e+08,600025,2025-01-03 06:09:31.993927,2024-12-05
338,1692,2022-07-14,7.05,9.56,9.68,6.82,1.770236e+08,600025,2025-01-03 06:09:31.993927,2024-12-11
339,1697,2024-12-12,9.55,9.71,9.95,9.53,3.224938e+08,600025,2025-01-03 06:09:31.993927,2024-12-18
340,1702,2024-12-19,9.66,9.89,9.92,9.33,2.934496e+08,600025,2025-01-03 06:09:31.993927,2024-12-25


In [12]:
df_raw.sort_values("date").tail(30)

,date,open,close,high,low,transaction_volume,stock_index,update_time
1681,2024-11-21,9.61,9.67,9.72,9.60,173074704.0,600025,2025-01-03 06:09:31.993927
1682,2024-11-22,9.69,9.42,9.74,9.41,302537674.0,600025,2025-01-03 06:09:31.993927
1683,2024-11-25,9.43,9.30,9.52,9.27,280021876.0,600025,2025-01-03 06:09:31.993927
1684,2024-11-26,9.30,9.23,9.35,9.21,169128082.0,600025,2025-01-03 06:09:31.993927
1685,2024-11-27,9.22,9.43,9.47,9.22,348919901.0,600025,2025-01-03 06:09:31.993927
1686,2024-11-28,9.46,9.32,9.46,9.31,171051080.0,600025,2025-01-03 06:09:31.993927
1687,2024-11-29,9.35,9.30,9.38,9.30,194378707.0,600025,2025-01-03 06:09:31.993927
1688,2024-12-02,9.32,9.40,9.41,9.27,249945549.0,600025,2025-01-03 06:09:31.993927
1689,2024-12-03,9.41,9.58,9.59,9.37,386491530.0,600025,2025-01-03 06:09:31.993927
1690,2024-12-04,9.57,9.64,9.65,9.50,305551888.0,600025,2025-01-03 06:09:31.993927


In [14]:
start_from = df_raw.shape[0]%5
df1 = df_raw.iloc[start_from:].copy(deep=False).sort_values("date").reset_index()
df3 = get_weekly_price(df1)
get_stock_indicator(df3)

NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values
/home/davidyu/software/Anaconda/lib/python3.7/site-packages/stockstats.py:467: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  lambda x: np.fabs(x - x.mean()).mean())
/home/davidyu/software/Anaconda/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,open,close,high,low,stock_index,date_time,stock_date,macdh,cci,...,rsi_24,kdjk,kdjd,kdjj,boll_ub,boll_lb,macd,macds,wr_6,wr_10
0,2012-08-01,0.52,0.62,0.63,0.51,601398,2012-08-01,2012-08-01,0.000000,NaN,...,NaN,63.888889,54.629630,82.407407,NaN,NaN,0.000000,0.000000,8.333333,8.333333
1,2012-08-08,0.61,0.63,0.67,0.57,601398,2012-08-08,2012-08-08,0.000100,66.666667,...,100.000000,67.592593,58.950617,84.876543,0.639142,0.610858,0.000224,0.000125,25.000000,25.000000
2,2012-08-15,0.63,0.65,0.71,0.60,601398,2012-08-15,2012-08-15,0.000461,93.548387,...,100.000000,68.395062,62.098765,80.987654,0.663884,0.602783,0.000906,0.000445,30.000000,30.000000
3,2012-08-22,0.65,0.68,0.71,0.61,601398,2012-08-22,2012-08-22,0.001212,82.828283,...,100.000000,73.930041,66.042524,89.705075,0.697915,0.592085,0.002277,0.001066,15.000000,15.000000
4,2012-08-29,0.67,0.69,0.70,0.63,601398,2012-08-29,2012-08-29,0.001620,76.323988,...,100.000000,79.286694,70.457247,96.945588,0.714992,0.593008,0.003372,0.001752,10.000000,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,2024-11-12,6.13,6.00,6.17,6.00,601398,2024-11-12,2024-11-12,-0.024890,9.501436,...,59.852952,51.765979,53.399170,48.499598,6.520099,5.282901,0.245803,0.270693,79.518072,54.545455
598,2024-11-19,6.00,6.15,6.24,5.98,601398,2024-11-19,2024-11-19,-0.024473,30.477390,...,61.793307,53.794399,53.530913,54.321372,6.480568,5.417432,0.240102,0.264575,67.307692,42.148760
599,2024-11-26,6.13,6.16,6.18,6.05,601398,2024-11-26,2024-11-26,-0.024702,28.553837,...,61.921334,53.905746,53.655857,54.405524,6.456446,5.518554,0.233697,0.258400,57.142857,41.322314
600,2024-12-03,6.15,6.21,6.23,6.10,601398,2024-12-03,2024-12-03,-0.022716,46.081505,...,62.575623,51.198208,52.836641,47.921343,6.445449,5.598551,0.230005,0.252721,11.538462,41.284404


In [11]:
df1

,index,date,open,close,high,low,transaction_volume,stock_index,update_time
0,0,2012-07-26,0.52,0.52,0.53,0.51,5.768488e+07,601398,2024-12-11 02:09:05.003376
1,1,2012-07-27,0.53,0.57,0.57,0.52,1.085801e+08,601398,2024-12-11 02:09:05.003376
2,2,2012-07-30,0.57,0.56,0.60,0.55,9.346376e+07,601398,2024-12-11 02:09:05.003376
3,3,2012-07-31,0.57,0.60,0.61,0.56,8.729492e+07,601398,2024-12-11 02:09:05.003376
4,4,2012-08-01,0.60,0.62,0.63,0.59,9.397817e+07,601398,2024-12-11 02:09:05.003376
...,...,...,...,...,...,...,...,...,...
3005,3005,2024-12-04,6.21,6.26,6.28,6.17,2.228037e+09,601398,2024-12-11 02:09:05.003376
3006,3006,2024-12-05,6.25,6.26,6.33,6.23,1.659231e+09,601398,2024-12-11 02:09:05.003376
3007,3007,2024-12-06,6.26,6.32,6.34,6.25,1.968099e+09,601398,2024-12-11 02:09:05.003376
3008,3008,2024-12-09,6.30,6.32,6.37,6.28,1.982869e+09,601398,2024-12-11 02:09:05.003376


In [7]:

def get_data_sql_str_macd():
    return f"""SELECT *  FROM t_stock_kdj_daily_last """

# t_stock_kdj_weekly_last
conn = CommonScript.connect_to_db("test.db")
cursor = conn.cursor()
df1 = pd.read_sql_query(get_data_sql_str_macd(),conn)
print(df1)

           date    open   close    high     low  stock_index   date_time  \
0    2024-11-22    7.22    6.99    7.26    6.97       600011  2024-11-22   
1    2024-11-22   24.94   24.23   24.98   24.22          895  2024-11-22   
2    2024-11-22    1.95    1.94    2.00    1.90       600010  2024-11-22   
3    2024-11-22   57.06   56.01   57.19   56.01       600660  2024-11-22   
4    2024-11-22    6.50    6.42    6.60    6.40       601728  2024-11-22   
..          ...     ...     ...     ...     ...          ...         ...   
246  2024-11-22    6.26    6.13    6.35    6.13          877  2024-11-22   
247  2024-11-22    7.92    7.50    7.93    7.48       600061  2024-11-22   
248  2024-11-22  141.64  136.29  142.00  136.18          568  2024-11-22   
249  2024-11-22   41.05   38.92   41.50   38.83       600584  2024-11-22   
250  2024-11-22   42.12   41.08   43.88   41.00         2460  2024-11-22   

     stock_date  macdh      cci  ...    kdjk    kdjd    kdjj  boll_ub  \
0    2024-11-2

In [10]:
df1["stock_index"] =df1["stock_index"].astype(str)

In [12]:
pipeline = pyg.pipeline()

AttributeError: module 'pygwalker' has no attribute 'pipeline'

In [11]:
import pygwalker as pyg
gwalker = pyg.walk(df1)   

Box(children=(HTML(value='\n<div id="ifr-pyg-000627900250cc7akc4GAJWMZlbgsoFh" style="height: auto">\n    <hea…